**Caso de uso: Como mejorar con embeddings - PARTE 3**

In [1]:
import torch
import pandas as pd
from torch import nn
from transformers import BartTokenizer, BartModel
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

messages = [
    "We have released a new product, do you want to buy it?",
    "Winner! Great deal, call us to get this product for free",
    "Tomorrow is my birthday, do you come to the party?"
]

In [2]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
bart_model = BartModel.from_pretrained("facebook/bart-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [3]:
df = pd.read_csv("/content/drive/My Drive/spam/data/SMSSpamCollection",
                 sep="\t",
                 names=["type", "message"])
df["spam"] = df["type"] == "spam"
df.drop("type", axis=1, inplace=True)
print(df.head())
print(df.shape)

                                             message   spam
0  Go until jurong point, crazy.. Available only ...  False
1                      Ok lar... Joking wif u oni...  False
2  Free entry in 2 a wkly comp to win FA Cup fina...   True
3  U dun say so early hor... U c already then say...  False
4  Nah I don't think he goes to usf, he lives aro...  False
(5572, 2)


In [4]:
df_train = df.sample(frac=0.8, random_state=0)
print(df_train.shape)
df_val = df.drop(index=df_train.index)
print(df_val.shape)

def convert_to_embeddings(messages):
  embeddings_list = []
  for message in tqdm(messages):
    out = tokenizer([message],
                    padding=True,
                    max_length=512,
                    truncation=True,
                    return_tensors="pt"
                    )
    with torch.no_grad():
      bart_model.eval()

      pred = bart_model(
          input_ids=out["input_ids"],
          attention_mask=out["attention_mask"]
          )
      embeddings = pred.last_hidden_state.mean(dim=1).reshape((-1))
      embeddings_list.append(embeddings)

  return torch.stack(embeddings_list)


(4458, 2)
(1114, 2)


In [5]:

X_train = convert_to_embeddings(df_train["message"].tolist())
X_val = convert_to_embeddings(df_val["message"].tolist())



100%|██████████| 1114/1114 [03:14<00:00,  5.72it/s]


In [6]:
y_train = torch.tensor(df_train["spam"].values, dtype=torch.float32).reshape((-1, 1))
y_val = torch.tensor(df_val["spam"].values, dtype=torch.float32).reshape((-1, 1))

In [ ]:
model = nn.Linear(768, 1)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.02)

In [ ]:
for i in range(0, 10000):
    # Training pass
    optimizer.zero_grad()
    outputs = model(X_train)
    print(outputs.shape)
    print(y_train.shape)
    loss = loss_fn(outputs, y_train)
    loss.backward()
    optimizer.step()

    if i % 1000 == 0:
        print(loss)

Streaming output truncated to the last 5000 lines.
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
torch.Size([4458, 1])
tor

In [ ]:
def evaluate_model(X, y):
    model.eval()
    with torch.no_grad():
        y_pred = nn.functional.sigmoid(model(X)) > 0.25
        print("accuracy:", (y_pred == y)\
            .type(torch.float32).mean())

        print("sensitivity:", (y_pred[y == 1] == y[y == 1])\
            .type(torch.float32).mean())

        print("specificity:", (y_pred[y == 0] == y[y == 0])\
            .type(torch.float32).mean())

        print("precision:", (y_pred[y_pred == 1] == y[y_pred == 1])\
            .type(torch.float32).mean())

print("Evaluating on the training data")
evaluate_model(X_train, y_train)

print("Evaluating on the validation data")
evaluate_model(X_val, y_val)

Evaluating on the training data
accuracy: tensor(0.9984)
sensitivity: tensor(0.9984)
specificity: tensor(0.9984)
precision: tensor(0.9902)
Evaluating on the validation data
accuracy: tensor(0.9946)
sensitivity: tensor(0.9856)
specificity: tensor(0.9959)
precision: tensor(0.9716)


In [ ]:
X_custom = convert_to_embeddings([
    "We have released a new feature for your product and you have been selected to try it!",
    "We have released a new product to improve your sales, do you want to try it",
    "Winner! Great deal, call us to get this product for free",
    "Tomorrow is my birthday, do you come to the party?"
])
model.eval()
with torch.no_grad():
    pred = nn.functional.sigmoid(model(X_custom))
    print(pred)

100%|██████████| 4/4 [00:00<00:00,  6.37it/s]

tensor([[8.2507e-01],
        [2.4644e-01],
        [7.5393e-01],
        [2.5175e-07]])
